<a href="https://colab.research.google.com/github/ivanDourado/Pyspark_analise_de_sentimentos_projeto/blob/main/AP_bigbata_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
RU = 3357756
# add arquivo no colab
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Remover cópias anteriores do arquivo Spark
!rm -f spark-3.4.2-bin-hadoop*.tgz*

# Baixar Spark
!wget https://dlcdn.apache.org/spark/spark-3.4.2/spark-3.4.2-bin-hadoop3.tgz

# Listar arquivos para verificar se o Spark foi baixado
!ls

# Extrair o Spark, se o arquivo estiver presente
!tar xf spark-3.4.2-bin-hadoop3.tgz || echo "O arquivo não foi baixado corretamente."

# Configurar as variáveis de ambiente para o Spark e Java
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.2-bin-hadoop3"


--2024-01-02 20:55:22--  https://dlcdn.apache.org/spark/spark-3.4.2/spark-3.4.2-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388664780 (371M) [application/x-gzip]
Saving to: ‘spark-3.4.2-bin-hadoop3.tgz’

spark-3.4.2-bin-had 100%[===================>] 370.66M  70.7MB/s    in 5.9s    

2024-01-02 20:55:32 (62.6 MB/s) - ‘spark-3.4.2-bin-hadoop3.tgz’ saved [388664780/388664780]

drive  imdb-reviews-pt-br.csv  sample_data  spark-3.4.2-bin-hadoop3  spark-3.4.2-bin-hadoop3.tgz


In [ ]:
# Listar o conteúdo do diretório para verificar a presença da pasta do Spark
!ls -l


total 503664
drwx------  5 root root      4096 Jan  2 17:29 drive
-rw-r--r--  1 root root 127066935 Jan  2 20:54 imdb-reviews-pt-br.csv
drwxr-xr-x  1 root root      4096 Dec 19 14:20 sample_data
drwxr-xr-x 13 1000 1000      4096 Nov 25 07:48 spark-3.4.2-bin-hadoop3
-rw-r--r--  1 root root 388664780 Nov 25 07:49 spark-3.4.2-bin-hadoop3.tgz


In [ ]:
# Instalar Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null



In [ ]:
# instalar pyspark
!pip install pyspark


In [ ]:

#import findspark
#findspark.init()

from pyspark.sql import SparkSession

# Iniciar uma SparkSession
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("Colab_Spark_Session")\
        .getOrCreate()


In [ ]:
#criando cópia do dataframe na pasta drive
#!cp /content/drive/MyDrive/ciencia_de_dados_Uninter/AP-bigdata/imdb-reviews-pt-br.csv /content

## Pratica 01 - Somatório de IDs.
Qual o valor da soma de todos os campos “id” dos filmes classificados como negativos para o banco de dados “imdb-reviews-pt-br.csv”

In [ ]:
# carregando o conjunto de dados

df= spark.read.csv(
    "/content/imdb-reviews-pt-br.csv",
    header=True, quote="\"",
    escape="\"", encoding="UTF-8"
    )

#Mostrando as 20 primiras linhas
df.show()

+---+--------------------+--------------------+---------+
| id|             text_en|             text_pt|sentiment|
+---+--------------------+--------------------+---------+
|  1|Once again Mr. Co...|Mais uma vez, o S...|      neg|
|  2|This is an exampl...|Este é um exemplo...|      neg|
|  3|First of all I ha...|Primeiro de tudo ...|      neg|
|  4|Not even the Beat...|Nem mesmo os Beat...|      neg|
|  5|Brass pictures mo...|Filmes de fotos d...|      neg|
|  6|A funny thing hap...|Uma coisa engraça...|      neg|
|  7|This German horro...|Este filme de ter...|      neg|
|  8|Being a long-time...|Sendo um fã de lo...|      neg|
|  9|"Tokyo Eyes" tell...|"Tokyo Eyes" fala...|      neg|
| 10|Wealthy horse ran...|Fazendeiros ricos...|      neg|
| 11|Cage plays a drun...|Cage interpreta u...|      neg|
| 12|First of all, I w...|Primeiro de tudo,...|      neg|
| 13|So tell me - what...|Então me diga - q...|      neg|
| 14|A big disappointm...|Uma grande decepç...|      neg|
| 15|This film

[Row(id='49459', text_en='A Christmas Together actually came before my time, but Ive been raised on John Denver and the songs from this special were always my familys Christmas music. For years we had a crackling cassette made from a record that meant it was Christmas. A few years ago, I was finally able to track down a video of it on Ebay, so after listening to all the music for some 21 years, I got to see John and the Muppets in action for myself. If you ever get the chance, its a lot of fun--great music, heart-warming and cheesy. Its also interesting to see the 70s versions of the Muppets and compare them to their newer versions today. I believe Denver actually took some heat for doing a show like this--I guess normally performers dont compromise their images by doing sing-a-longs with the Muppets, but Im glad he did. Even if you cant track down the video, the soundtrack is worth it too. It has some Muppified traditional favorites, but also some original Denver tunes as well.', text

In [ ]:
# print do shema
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- text_en: string (nullable = true)
 |-- text_pt: string (nullable = true)
 |-- sentiment: string (nullable = true)



É preciso tratar a coluna ID pois essa é do tipo string. E é necessário convertê-la para INT de moo a efetuar operações matemáticas.

In [ ]:
#confirmando tipo do dataframe
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
# contando as linhas do conjunto de dados

print(f'O número de linhas do dataframe é de {df.count()} linhas.')


O número de linhas do dataframe é de 49459 linhas.


In [ ]:
df.describe()

DataFrame[summary: string, id: string, text_en: string, text_pt: string, sentiment: string]

In [ ]:
# importando métodos com funções com fito de converter variáveis
from pyspark.sql.functions import  *

# convertendo coluna ID de string para INT
df= df.withColumn(
    colName='id',
    col=regexp_replace('id','','').cast('int')
)

# printando o resultado
df.select('id').printSchema()



root
 |-- id: integer (nullable = true)



In [ ]:
# criando dataframe apenas com as colunas sentiment e ID
df_select = df.select('id','sentiment')

# Mostrando o resultado
df_select.show()

df_select.count()

+---+---------+
| id|sentiment|
+---+---------+
|  1|      neg|
|  2|      neg|
|  3|      neg|
|  4|      neg|
|  5|      neg|
|  6|      neg|
|  7|      neg|
|  8|      neg|
|  9|      neg|
| 10|      neg|
| 11|      neg|
| 12|      neg|
| 13|      neg|
| 14|      neg|
| 15|      neg|
| 16|      neg|
| 17|      neg|
| 18|      neg|
| 19|      neg|
| 20|      neg|
+---+---------+
only showing top 20 rows



49459

In [ ]:
# Criando DF somente com os valores da colun 'sentiment' = 'neg'
df_neg= df_select.where('sentiment == "neg"').orderBy('id')

#exibindo resultado
print(f'Acima, 20 primeiras linhas do dataframe com comentarios classificados como negativos.')
df_neg.show()
df_neg.count()


Acima, 20 primeiras linhas do dataframe com comentarios classificados como negativos.
+---+---------+
| id|sentiment|
+---+---------+
|  1|      neg|
|  2|      neg|
|  3|      neg|
|  4|      neg|
|  5|      neg|
|  6|      neg|
|  7|      neg|
|  8|      neg|
|  9|      neg|
| 10|      neg|
| 11|      neg|
| 12|      neg|
| 13|      neg|
| 14|      neg|
| 15|      neg|
| 16|      neg|
| 17|      neg|
| 18|      neg|
| 19|      neg|
| 20|      neg|
+---+---------+
only showing top 20 rows



24765

In [ ]:
# Somatório dos valores da coluna ID cujos valores da coluna sentiment == neg

total_neg = df_neg.groupBy('sentiment').agg(sum('id').alias("Total_Somatório_Neg")).orderBy('sentiment').where('sentiment =="neg"')

total_neg.show()

total_neg.count()

+---------+-------------------+
|sentiment|Total_Somatório_Neg|
+---------+-------------------+
|      neg|          459568555|
+---------+-------------------+



1

In [ ]:
# printando resultado
RU = 3357756
soma_neg = total_neg.select(col("Total_Somatório_Neg")).first()[0]

print(f'Valor do somatório dos IDs negativos é de {soma_neg}.\nRU do aluno:{RU}')

Valor do somatório dos IDs negativos é de 459568555.
RU do aluno:3357756


### Valor do somatório dos IDs negativos é de 459568555.





---------------------------------------------------
#Prática 02 - Diferença do número de palavras totais de português para inglês dos textos negativos


Contar palavras dos textos negativos e achar diferença de quantidade.

In [ ]:
# selecionando uma coluna pra cada idioma
RU = 3357756

df_en = df.select('id','text_en','sentiment')
df_pt = df.select('id','text_pt','sentiment')

In [ ]:
# filtrando o conjunto de dados para o sentiment = neg

df_en = df_en.where('sentiment == "neg"')
df_pt = df_pt.where('sentiment == "neg"')

#removendo a coluna sentiment
df_en = df_en.drop('sentiment')
df_pt = df_pt.drop('sentiment')

df_en.show()
df_pt.show()

+---+--------------------+
| id|             text_en|
+---+--------------------+
|  1|Once again Mr. Co...|
|  2|This is an exampl...|
|  3|First of all I ha...|
|  4|Not even the Beat...|
|  5|Brass pictures mo...|
|  6|A funny thing hap...|
|  7|This German horro...|
|  8|Being a long-time...|
|  9|"Tokyo Eyes" tell...|
| 10|Wealthy horse ran...|
| 11|Cage plays a drun...|
| 12|First of all, I w...|
| 13|So tell me - what...|
| 14|A big disappointm...|
| 15|This film is abso...|
| 16|Heres a decidedly...|
| 17|At the bottom end...|
| 18|Earth has been de...|
| 19|Many people are s...|
| 20|New York family i...|
+---+--------------------+
only showing top 20 rows

+---+--------------------+
| id|             text_pt|
+---+--------------------+
|  1|Mais uma vez, o S...|
|  2|Este é um exemplo...|
|  3|Primeiro de tudo ...|
|  4|Nem mesmo os Beat...|
|  5|Filmes de fotos d...|
|  6|Uma coisa engraça...|
|  7|Este filme de ter...|
|  8|Sendo um fã de lo...|
|  9|"Tokyo Eyes" fala...|
| 

In [ ]:
# dividindo o texto em palavras com as funções explode e split
palavras_pt = df_pt.select("id", explode(split("text_pt", "[ ]+")).alias("palavra"))
palavras_en = df_en.select("id", explode(split("text_en", "[ ]+")).alias("palavra"))

# Contar a ocorrencia de cada palavra de forma ordenada
contagem_pt = palavras_pt.groupBy("palavra").count().orderBy(desc("count"))
contagem_en = palavras_en.groupBy("palavra").count().orderBy(desc("count"))

In [ ]:
# Exibe resultado
print(f"Palavras em Português.\nRU:{RU}")
contagem_pt.show()

print(f"Palavras em Inglês.\nRU:{RU}")
contagem_en.show()

Palavras em Português.
RU:3357756
+-------+------+
|palavra| count|
+-------+------+
|     de|209113|
|    que|167678|
|      e|137200|
|      o|123049|
|     um|107492|
|      a|102794|
|      é| 93327|
|    não| 73713|
|    uma| 64633|
|   para| 64105|
|     em| 63829|
|  filme| 61801|
|     do| 49849|
|    com| 48684|
|     eu| 37822|
|     se| 37108|
|   como| 36659|
|     os| 36428|
|    mas| 33143|
|    por| 31171|
+-------+------+
only showing top 20 rows

Palavras em Inglês.
RU:3357756
+-------+------+
|palavra| count|
+-------+------+
|    the|265718|
|      a|145812|
|    and|131834|
|     of|128764|
|     to|127790|
|     is| 91404|
|     in| 76328|
|      I| 69703|
|   that| 61670|
|   this| 60545|
|     it| 53013|
|    was| 48979|
|    for| 38483|
|   with| 37810|
|     as| 35494|
|    The| 34522|
|  movie| 33879|
|    but| 32892|
|     on| 29428|
|   have| 28906|
+-------+------+
only showing top 20 rows



In [ ]:
#Soma dos valores
contagem_pt.agg(sum("count").alias("Total_palavras_pt")).show()
contagem_en.agg(sum("count").alias("Total_palavras_en")).show()

+-----------------+
|Total_palavras_pt|
+-----------------+
|          5455273|
+-----------------+

+-----------------+
|Total_palavras_en|
+-----------------+
|          5400276|
+-----------------+



In [ ]:
# coletando os valores para efetuar a subtração
tot_pt = contagem_pt.agg(sum("count").alias("Total_palavras_pt")).first()[0]
tot_en = contagem_en.agg(sum("count").alias("Total_palavras_en")).first()[0]

tot_pt, tot_en

(5455273, 5400276)

In [ ]:
# printando a resposta
print(f'''O texto em português possui {tot_pt} palavras.
\nO Texto em inglês possui {tot_en} palavras.
\nLogo o texto em português possui {tot_pt - tot_en} palavras a mais queo texto em inglês.''')

O texto em português possui 5455273 palavras. 

O Texto em inglês possui 5400276 palavras.

Logo o texto em português possui 54997 palavras a mais queo texto em inglês.


#Logo o texto em português possui 54997 palavras a mais queo texto em inglês.
